# SKEMPI 2.0 benchmark: data filtraton and file structure preparation

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
#import Bio.PDB
import os
import shutil
import Bio

'inputs_pdb' is the folder where all case subdirectories are. <br/>
Subdirectories are called by case (filtrated SKEMPI dataframe index plus PDB ID) and contain a superlink to the cleaned PDB file in the main directory and a resfile for every case. This structure is important for Flex ddG initiating run_DeiC.py, which is submitted as a SLURM job by Flex_ddG_SLURM.ipynb notebook.

In [1]:
inputs='/groups/sbinlab/panf/SKEMPIv2_benchmark/pdbs_master'
main = os.path.join(inputs, "PDBs")

NameError: name 'os' is not defined

In [5]:
# Load dataset and get list of mutations in cases and unique PDB list 
# (for downloading PDBs from PDB website)

skempi2 = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2/skempi_v2.csv', sep=';')
skempi2.drop_duplicates(inplace=True)

#skempi2 = skempi2.loc[~skempi2['Affinity_mut (M)'].str.startswith(('<', '>'))]
#skempi2 = skempi2[skempi2['Method'].isin(['ITC', 'SPR', 'FL', 'SP'])].copy().reset_index(drop=True)
#skempi2 = skempi2.loc[skempi2['Affinity_mut_parsed']!=skempi2['Affinity_wt_parsed']].reset_index(drop=True)

skempi2['dG_bind_MUT'] = 0.001987204 * skempi2['Temperature'].str.extract('(\d+)'
                        ).values.astype(float).reshape(-1) * np.log(skempi2['Affinity_mut_parsed'].values)
skempi2['dG_bind_WT'] =  0.001987204 * skempi2['Temperature'].str.extract('(\d+)'
                        ).values.astype(float).reshape(-1) * np.log(skempi2['Affinity_wt_parsed'].values)
#skempi2['ddG_bind'] = skempi2['dG_bind_MUT'].values - skempi2['dG_bind_WT'].values

groups = skempi2[['#Pdb', 'Mutation(s)_PDB', 'dG_bind_MUT', 'dG_bind_WT']].groupby(['#Pdb', 'Mutation(s)_PDB'])
group_mean = groups.mean()
group_std = groups.std()
group_sizes = groups.size()
stds = pd.merge(skempi2, group_std, on=['#Pdb', 'Mutation(s)_PDB'], suffixes=('', '_std'))
stds.drop(columns=['dG_bind_MUT', 'dG_bind_WT'], inplace=True)

stds['dG_bind_MUT_std'].fillna(0, inplace=True)
stds['dG_bind_WT_std'].fillna(0, inplace=True)
low_err = stds.loc[((stds['dG_bind_MUT_std']<0.5)&(stds['dG_bind_WT_std']<0.5))].copy()
skempi2 = pd.merge(low_err, group_mean, on=['#Pdb', 'Mutation(s)_PDB'])
skempi2['ddG_bind'] = skempi2['dG_bind_MUT'].values - skempi2['dG_bind_WT'].values
skempi2.drop_duplicates(subset=['#Pdb', 'Mutation(s)_PDB', 'ddG_bind'], inplace=True)
skempi2.drop(columns=['dG_bind_MUT_std', 'dG_bind_WT_std'], inplace=True)

skempi2['PDB_ID'] = skempi2['#Pdb'].str.split("_").str[0]
skempi2['LChains'] = skempi2['#Pdb'].str.split("_").str[1]
skempi2['RChains'] = skempi2['#Pdb'].str.split("_").str[2]
skempi2["Chains"] = skempi2["LChains"].astype(str) + skempi2["RChains"].astype(str)

# Get a list of PDBs for which we use different chains for different entries
dif_chains = skempi2[['PDB_ID', 'Chains']].groupby(['PDB_ID', 'Chains']).size().reset_index()
dif_chains_counts = dif_chains['PDB_ID'].value_counts()
dif_chains_PDBs = dif_chains_counts[dif_chains_counts > 1].reset_index()['PDB_ID'].values.tolist()

# Drop those PDBs
skempi2 = skempi2.loc[~skempi2['PDB_ID'].isin(dif_chains_PDBs)].reset_index(drop=True)

# Alternativery, keep all the chains
# for pdb in dif_chains_PDBs:
#     lst = set()
#     for set_of_chains in dif_chains.loc[dif_chains['PDB_ID']==pdb].Chains:
#         lst = lst.union(set(list(set_of_chains)))
#     skempi2.loc[skempi2['PDB_ID']==pdb, 'Chains'] = ''.join(list(lst))

skempi2['index'] = skempi2.index.tolist()
skempi2['case_ID'] = skempi2['index'].astype('str') + '_' + skempi2['PDB_ID']

In [190]:
#skempi2.to_csv('/groups/sbinlab/panf/SKEMPIv2/skempi2_mesure_mean.csv', index=False)

In [6]:
dif_chains_PDBs

['3SE3', '3SE4', '2C5D']

In [177]:
skempi2_new = skempi2.drop(columns=['index'])

In [178]:
skempi2_old = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2/skempi2_filt.csv')
skempi2_old.drop(columns=['Unnamed: 0', 'PDBFileID', 'case_name', 'dG_bind_MUT_std', 'dG_bind_WT_std'], inplace=True)

In [179]:
df_all = pd.merge(skempi2_new, skempi2_old.drop_duplicates(subset=['#Pdb', 'Mutation(s)_PDB', 'ddG_bind']),
                  on=['#Pdb', 'Mutation(s)_PDB'], how='left', indicator=True)

In [180]:
df_all['_merge'].value_counts()

_merge
both          4039
left_only     1849
right_only       0
Name: count, dtype: int64

In [187]:
new_cases = df_all.loc[df_all['_merge']=='left_only']['case_ID'].tolist()
lst_new_cases = open('list_new_cases.txt', 'w+')
for new_case in new_cases:
    lst_new_cases.write(new_case + '\r\n')
lst_new_cases.close()

In [128]:
# FOR FUTUTRE ANALYSIS: No binding detected, I would guess predictions should be >15 kcal/mol
# skempi2_filt.loc[skempi2_filt['Affinity_mut (M)']=='n.b']
# skempi2.loc[skempi2['Affinity_mut (M)'] == 'n.b', 'dG_bind_MUT'] = 100

In [35]:
# Get a list of uniqie PDB IDs in the dataset and write it to a file to use with PDB batch_download.sh

# PDB_IDs = skempi2['PDB_ID'].unique()
# lst = open('list_file.txt', 'w+')
# for id in PDB_IDs:
#     lst.write(id)
#     lst.write(',')
# lst.close()

In [134]:
# Remove PDBs that are mot in the dataset after filtration

for folder in os.listdir(main):
    if not folder in PDB_IDs and not folder == '.ipynb_checkpoints':
        shutil.rmtree(os.path.join(main, folder))

In [36]:
# CREATE SEPARATE FOLDER FOR EACH UNIQUE PDB_ID IN THE DATASET AND MOVE A .PDB FILE THERE

for folder in PDB_IDs:
    os.mkdir(os.path.join(main, folder))
    path_current = os.path.join(main, (folder + '.pdb'))
    path_new = os.path.join(main, folder, (folder + '.pdb'))
    shutil.move(path_current, path_new)

## Get resfiles

#### Example of resfile:

B T58D;A N92A;B T28D;B S57V<br>

should become:<br>

NATAA<br>
start<br>
58 B PIKAA D<br>
92 A PIKAA A<br>
28 B PIKAA D<br>
57 B PIKAA V<br>

In [139]:
# Get resfiles, which contain mutation position, chain, type and substitution aa 

## DOES NOT GIVE CHAINS TO MOVE; WRONG

for case_id, mutations in zip(skempi2['case_ID'], skempi2['Mutation(s)_PDB']):
    PBD_folder_path = os.path.join(main, case_id.split('_')[1])

    # Delete all resfiles
    # os.remove(os.path.join(PBD_folder_path, (case_name+'.resfile')))
    
    r = open(os.path.join(PBD_folder_path, (case_id+'.resfile')),"w+")
    r.write("NATAA \r\nstart \r\n")
    
    for mut in mutations.split(','):
        position = mut[2:-1].upper()
        chain = mut[1]
        aa = mut[-1]
        r.write(position + ' ' + chain + ' ' + 'PIKAA' + ' ' + aa + '\r\n')
    r.close()

## PDB files cleaning

In [135]:
#For initial pipeline
for folder in os.listdir(main):
    if not folder.endswith('.ipynb_checkpoints'):
        for file in os.listdir(os.path.join(main, folder)):
            if file.endswith('.pdb'):
                pdb_path = os.path.join(main, folder, file)
                PDB_ID = folder
                old_pdb = open(pdb_path, 'r')
                cleaned_pdb = open(os.path.join(main, folder, ('Cleaned_' + file)), 'w+')
                lines = old_pdb.readlines()
                chains = skempi2.loc[skempi2['PDB_ID']==PDB_ID, 'Chains'].unique()[0]
                chains_sep = list(chains)
                for line in lines:
                    if line.startswith('ATOM'):
                        if line[21] in chains_sep:
                            cleaned_pdb.write(line)
                cleaned_pdb.close()       

### Remove 'cleaned' PDBs from folders

In [78]:
for folder in os.listdir(main):
    if not folder.endswith('.ipynb_checkpoints'):
        for file in os.listdir(os.path.join(main, folder)):
            if file.startswith('Cleaned'):
                os.remove(os.path.join(main, folder, file))

## Creating in folder 'input' all required folders with symbolic links to PDBs and resfiles

In [ ]:
# Clear inputs
# for dir in os.listdir(inputs):
#     if not dir == 'main':
#         shutil.rmtree(os.path.join(inputs, dir))

In [140]:
for folder in os.listdir(main):
    if not folder.endswith(('.ipynb_checkpoints')):
        for file in os.listdir(os.path.join(main, folder)):
            if file.startswith('Cleaned_'):
                PDB = file
        for file in os.listdir(os.path.join(main, folder)):
            if file.endswith('.resfile'):
                index = file.split('_')[0]
                resfile = file
                src = os.path.join(main, folder)
                dst = os.path.join(inputs, (index + '_' + folder))
                os.makedirs(dst, exist_ok=True)
                os.symlink(os.path.join(src, resfile), os.path.join(dst, resfile))
                os.symlink(os.path.join(src, PDB), os.path.join(dst, PDB))